In [1]:
from es import ES
from read_pdf import read_pdf
from rag import build_prompt, get_completion

# 一、演示RAG

## 1、读取pdf

In [2]:
filename = "./data/基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究2020.pdf"
text_list = read_pdf(filepath=filename, min_line_length=10)

## 2、新建索引

In [3]:
index_name = "paper_db_cn"
es = ES(index_name=index_name)

## 3、往索引里添加数据

In [4]:
es.add_data_to_es(text_list)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/37/d9g7vr4j50s4mcyf2k_04d2r0000gn/T/jieba.cache
Loading model cost 0.596 seconds.
Prefix dict has been built successfully.


(426, [])

## 4、Prompt模版

In [5]:
prompt_template = """
已知信息:
__INFO__

用户问：
__QUERY__

请用中文回答用户问题。
"""

## 5、检索

In [9]:
user_query = "这篇论文中使用的是几层的LSTM？"
search_results = es.search(user_query, 3)

keywords 这篇 论文 几层 lstm
search_res {'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 134, 'relation': 'eq'}, 'max_score': 7.6296277, 'hits': [{'_index': 'paper_db_cn', '_id': 'dAxXD5ABwin7dsBfyv4U', '_score': 7.6296277, '_ignored': ['keywords.keyword', 'text.keyword'], '_source': {'keywords': '15 precisionrecall 训练 提出 两层 单向 长短期 记忆 神经网络 结构 一层 单向 结构 双向 结构 两层 双向 结构 分析 一层 单向 结构 隐藏 一层 正向 误差 累积 一层 双向 结构 隐藏 一层 模型 正向 误差 累积 反向 误差 累积 两层 双向 结构 隐藏 层有 两层 只进 正向 误差 累积 两层 双向 结构 隐藏 层有 两层 模型 正向 误差 累积 反向 误差 累积 从表 模型 训练 测试 集上 f1 分数 top1 确率 一层 单向 结构 一层 双向 结构 两层 单向 结构 模型 结构 相对 简单 导致 学习 充分 两层 双向 结构 两层 单向 结构 相当 两层 双向 模型 结构 相对 模型 数量 两层 单向 模型 模型 训练 过程 损失 函数 变化 两层 双向 模型 拟合 现象 本文 选择 两层 单向 模型 结构 细线 代表 训练 集上 损失 函数 变化 曲线 粗线 代表 验证 集上 损失 函数 变化 曲线 从图 两层 单向 长短期 记忆 神经网络 损失 函数 变化 曲线图 模型 迭代 收敛', 'text': '…(4.15) precision\uf02brecall 4.2.4.4 训练结果 把4.2.3.2节提出的两层单向长短期记忆神经网络结构与一层单向结构、一 层双向结构、两层双向结构进行对比分析。其中，一层单向结构，即隐藏层只 有一层，只进行正向误差累积；一层双向结构，即隐藏层只有一层

## 6、根据Prompt模版生成Prompt

In [10]:
prompt = build_prompt(prompt_template, info=search_results, query=user_query)
print("Prompt:\n", prompt)

Prompt:
 
已知信息:
…(4.15) precisionrecall 4.2.4.4 训练结果 把4.2.3.2节提出的两层单向长短期记忆神经网络结构与一层单向结构、一 层双向结构、两层双向结构进行对比分析。其中，一层单向结构，即隐藏层只 有一层，只进行正向误差累积；一层双向结构，即隐藏层只有一层，模型进行 正向误差累积，也进行反向误差累积；两层双向结构，即隐藏层有两层，只进 行正向误差累积；两层双向结构，即隐藏层有两层，模型进行正向误差累积， 也进行反向误差累积。从表4.2模型在训练集和测试集上的 F1分数和TOP1 准 确率来看，一层单向结构和一层双向结构均比两层单向结构的值低，此时由于 模型结构相对简单，导致学习不充分；两层双向结构和两层单向结构的值相当， 但是两层双向模型结构相对复杂，模型参数量是两层单向模型的 2.5 倍，且从 图4.9模型训练过程中损失函数变化来看，两层双向模型更易出现过拟合现象， 故本文选择两层单向模型结构。图 4.9 中，细线代表训练集上的损失函数变化 曲线，粗线代表验证集上的损失函数变化曲线。从图4.9(c)两层单向长短期记忆 神经网络损失函数变化曲线图可以看出，模型在第 3次迭代已经开始收敛，随
i1 (3) 词嵌入技术：词嵌入(word embedding)是一种把词映射为实数域向量的 技术，它将词汇表中的每一个词，表示成一个定长的向量，通过对特定任务的 学习，使得这些向量能较好地表达相似词之间的相似和类比关系。原始的文本 数据需要经过预处理成可以参与运算的数值数据才能输入网络，而词向量为文 本数据提供了一种数值化的表示方法，这是文本数据能够被计算机处理的基础， 也是深度学习技术能够应用于文本数据处理的重要前提基础。此外，使用词向 量技术还避免了独热编码在词库中词的数量较多时带来的数据冗余、运算效率 低，无法准确表达不同词之间的相似度等问题。图4.4是词汇数量为n，词向量 维度为K的表示。在数据预处理时，由于时间是不断变化的，很难用固定的量 来表示，因此把接收开始时间和接收结束时间属性分别拆成年、月、日、时、 分、秒、一年中的第几天、一周中的第几天八个属性列，除此之外数据接收任 务中还有卫星、地面站、任务优先级、任务紧急程度这四个属性，一共二十个 属性，每个属性均有若干个属性值，统计所有不重复出现的属性值，作为训练

## 7、调用大模型,参考检索结果，生成问题答案

In [11]:
response = get_completion(prompt)
print("答案：\n", response)

答案：
 这篇论文中使用的是两层的LSTM。


# 二、使用RAG,提问和我上传的这篇论文无关的问题

In [16]:
user_query = "InSAR技术都有哪些用途？"
search_results = es.search(user_query, 3)

keywords insar 技术 用途
search_res {'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 51, 'relation': 'eq'}, 'max_score': 9.708781, 'hits': [{'_index': 'paper_db_cn', '_id': 'NAxXD5ABwin7dsBfyv4U', '_score': 9.708781, '_ignored': ['keywords.keyword', 'text.keyword'], '_source': {'keywords': 'i1 嵌入 技术 嵌入 word embedding 一种 映射 实数 向量 技术 词汇表 定长 向量 特定 学习 向量 表达 之间 类比 关系 原始 文本 数据 预处理 参与 运算 数值数据 输入 网络 而词 向量 为文 数据 提供 一种 数值 方法 这是 文本 数据 计算机 基础 深度 学习 技术 文本 数据处理 前提 基础 词向 技术 独热 编码 词库 中词 数量 较多时 带来 数据 冗余 运算 效率 准确 表达 之间 词汇 数量 向量 维度 数据 预处理 时间 变化 难用 固定 接收 时间 接收 结束 时间 属性 一年 第几天 一周 第几天 八个 属性 数据 接收 务中 卫星 地面站 优先级 紧急 程度 四个 属性 一共 二十个 属性 属性 若干个 属性 统计 重复 属性 训练 数据 词库 词库 开是 整数 编码 嵌入 技术 编码 映射 实数 特定 维度 向量 数据 接收 映射 成特 维度 向量 向量 lstm 网络 时间 输入 向量 lstm 模型 训练 过程 调整 lstm 模型 训练 稳定', 'text': 'i\uf02b1 (3) 词嵌入技术：词嵌入(word embedding)是一种把词映射为实数域向量的 技术，它将词汇表中的每一个词，表示成一个定长的向量，通过对特定任务的 学习，使得这些向量能较好地表达相似词之间的相似和类比关系。原始的文本 数据需要经过预处理成可以参与运算的数值数据才能输入网络，而词向量为文 本数据提

In [17]:
prompt = build_prompt(prompt_template, info=search_results, query=user_query)
print("Prompt:\n", prompt)

Prompt:
 
已知信息:
i1 (3) 词嵌入技术：词嵌入(word embedding)是一种把词映射为实数域向量的 技术，它将词汇表中的每一个词，表示成一个定长的向量，通过对特定任务的 学习，使得这些向量能较好地表达相似词之间的相似和类比关系。原始的文本 数据需要经过预处理成可以参与运算的数值数据才能输入网络，而词向量为文 本数据提供了一种数值化的表示方法，这是文本数据能够被计算机处理的基础， 也是深度学习技术能够应用于文本数据处理的重要前提基础。此外，使用词向 量技术还避免了独热编码在词库中词的数量较多时带来的数据冗余、运算效率 低，无法准确表达不同词之间的相似度等问题。图4.4是词汇数量为n，词向量 维度为K的表示。在数据预处理时，由于时间是不断变化的，很难用固定的量 来表示，因此把接收开始时间和接收结束时间属性分别拆成年、月、日、时、 分、秒、一年中的第几天、一周中的第几天八个属性列，除此之外数据接收任 务中还有卫星、地面站、任务优先级、任务紧急程度这四个属性，一共二十个 属性，每个属性均有若干个属性值，统计所有不重复出现的属性值，作为训练 数据集的词库。对词库中的词从 0 开是进行整数编码，再使用词嵌入技术，把 每个编码映射为实数域特定维度的词向量，相应的每个数据接收任务映射成特 定维度的任务向量，此向量作为LSTM网络每个时间步的输入。每个词对应的 词向量，在LSTM模型训练过程中会不断调整，当LSTM模型训练稳定时，词
站资源调度。 2011年09月-2015年07月，在上海师范大学地理系获得学士学位。 2015年12月-2016年09月，就职于上海飞未信息技术有限公司，担任WebGIS
数据项 含义 值 1 卫星名称 GF-5 2 地面接收站 SYC 3 接收开始时间 2019-09-1314:56:51 4 接收结束时间 2019-09-1315:04:53 5 任务紧急程度 一般 6 任务优先级 0.5 7 天线 SY12_4 4.2.2 数据预处理 深度学习模型训练过程中，数据集的大小和质量直接影响模型的训练以及 模型效果，因此在模型训练之前，做好数据预处理工作至关重要。论文中主要 使用了三个数据预处理技术，分别为数据缺失处理、词嵌入技术、训练集和验 证集的划分方法，数据预处理流程如图4.3所示。

用户问：
InSAR技术都有哪些用

In [18]:
response = get_completion(prompt)
print("答案：\n", response)

答案：
 我无法回答您的问题。


# 三、不是用RAG，提问和这篇论文相关的问题

In [19]:
prompt_template = """
已知信息:
__INFO__

用户问：
__QUERY__

请用中文回答用户问题。
"""

In [20]:
user_query = "这篇论文中使用的是几层的LSTM？"
# search_results = es.search(user_query, 3)

In [21]:
prompt = build_prompt(prompt_template, query=user_query)
print("Prompt:\n", prompt)

Prompt:
 
已知信息:
__INFO__

用户问：
这篇论文中使用的是几层的LSTM？

请用中文回答用户问题。



In [22]:
response = get_completion(prompt)
print("答案：\n", response)

答案：
 我无法回答您的问题。
